In [6]:
!pip install -U transformers


[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [6]:
import pandas as pd
from trl import SFTTrainer
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Pipeline,
    PreTrainedModel,
    PreTrainedTokenizerBase,
    pipeline,
)
import os

tokenizer = AutoTokenizer.from_pretrained("unsloth/Mistral-Small-Instruct-2409")

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/588k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

In [9]:
all_datasets = ['DFINAL_VTrain.csv', 'DFINAL_VTest.csv']

In [10]:
def mapper(x):
    clean_chat = []
    x = eval(x)
    for ll in x:
        if len(ll.keys()) != 2:
            clean_chat.append({
                'role': ll['role'],
                'content': ll['content'],
            })
        else:
            clean_chat.append(ll)
    return clean_chat

In [12]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "mistral",
)

AssertionError: Torch not compiled with CUDA enabled

In [11]:
for dataset in all_datasets:
    path = os.path.join('../../data/training_data', dataset)
    df = pd.read_csv(path)
    df['chat'] = df.chat.map(lambda x: mapper(x))
    df["text"] = df.chat.map(lambda x: tokenizer.apply_chat_template(conversation=x, tokenize=False))
    new_path = os.path.join('../../data/training_data', dataset.split('.')[0] + '_mistral-small.csv')
    df.to_csv(new_path, index=False)

TemplateError: After the optional system message, conversation roles must alternate user/assistant/user/assistant/...

In [4]:
df.head()

,Unnamed: 0,game,benchmark_version,game_id,model,experiment,episode,Aborted,Lose,Success,chat,text
0,0,imagegame,v1.6,0,o1-preview-2024-09-12-t0.0--o1-preview-2024-09...,1_random_grids,episode_0,0,0,1,"[{'role': 'user', 'content': 'Let us do the fo...","<|im_start|>system\nYou are Qwen, created by A..."
1,1,imagegame,v1.6,0,o1-preview-2024-09-12-t0.0--o1-preview-2024-09...,1_random_grids,episode_0,0,0,1,"[{'role': 'user', 'content': 'Let us do the fo...","<|im_start|>system\nYou are Qwen, created by A..."
2,2,imagegame,v1.6,0,o1-preview-2024-09-12-t0.0--o1-preview-2024-09...,1_random_grids,episode_0,0,0,1,"[{'role': 'user', 'content': 'Let us do the fo...","<|im_start|>system\nYou are Qwen, created by A..."
3,3,imagegame,v1.6,0,o1-preview-2024-09-12-t0.0--o1-preview-2024-09...,1_random_grids,episode_0,0,0,1,"[{'role': 'user', 'content': 'Let us do the fo...","<|im_start|>system\nYou are Qwen, created by A..."
4,4,imagegame,v1.6,0,o1-preview-2024-09-12-t0.0--o1-preview-2024-09...,1_random_grids,episode_0,0,0,1,"[{'role': 'user', 'content': 'Let us do the fo...","<|im_start|>system\nYou are Qwen, created by A..."


In [5]:
df.text.iloc[0]

'<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nLet us do the following task. The goal is to fill an empty grid that looks like this:\n\n▢ ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\n\nA filled grid below is 5 by 5 and can look like this:\n\n▢ ▢ ▢ ▢ ▢\n▢ ▢ E ▢ ▢\n▢ ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\nX X X X X\n\nI want you to describe this grid to me, step by step. You don\'t need to describe the empty squares, which are denoted with "▢". Only describe the location of letters in the grid. Then you wait for me to say "What is your next instruction?", and then you continue with the next step. Take the size of the grid into consideration while giving instructions. When you have described everything, you say "DONE".\n\nFor the filled grid above, here are the example steps.\n\nWhat is your next instruction?\nInstruction: Put an E in second row third column\n\nWhat is your next instruction?\nInstruction: Fill the last row with 

In [3]:
import pandas as pd
import os
from datasets import Dataset, load_dataset
df = pd.read_csv(os.path.join('../../data/training_data', 'DFINAL_qwen-coder.csv'))
ds: Dataset = Dataset.from_pandas(df=df)

In [36]:
for i,c in enumerate(list(df.chat)):
    if not isinstance(c, str):
        print(c, i)

In [55]:
for d in ds["chat"]:
    if not isinstance(d, str):
        print(d)

In [7]:
print(df.text.iloc[0])

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Let us draw something together. There is an empty grid with a size 5 by 5, like so:

▢ ▢ ▢ ▢ ▢
▢ ▢ ▢ ▢ ▢
▢ ▢ ▢ ▢ ▢
▢ ▢ ▢ ▢ ▢
▢ ▢ ▢ ▢ ▢

I will give you instructions like "put an X in the top left", and you return the grid by applying the given instruction, like so:

Instruction: put an X in the top left

X ▢ ▢ ▢ ▢
▢ ▢ ▢ ▢ ▢
▢ ▢ ▢ ▢ ▢
▢ ▢ ▢ ▢ ▢
▢ ▢ ▢ ▢ ▢

Or for another instruction such as "fill the fifth column with T", you return the updated grid by applying the given instruction in all places that the command corresponds to, like so:

Instruction: fill the fifth column with T

X ▢ ▢ ▢ T
▢ ▢ ▢ ▢ T
▢ ▢ ▢ ▢ T
▢ ▢ ▢ ▢ T
▢ ▢ ▢ ▢ T

Or for another instruction such as "fill the fourth column second row with P", you return the updated grid by applying the given instruction in all places that the command corresponds to, like so:

Instruction: fill the fourth column second row with P

X ▢ ▢ ▢ T
▢ ▢ ▢ P T
▢ ▢ ▢ ▢ T
▢ ▢ ▢